<a href="https://colab.research.google.com/github/PolMix/nems_ai/blob/main/Data%20Formatting/Data_Formatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lib import

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import seaborn as sns

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm

# Загрузка данных и просмотр их первоначальной структуры

Загружаем датасет с GitHub:

In [ ]:
import zipfile
import os

!wget --no-check-certificate \
    "https://github.com/PolMix/nems_ai/archive/refs/heads/main.zip" \
    -O "/tmp/data.zip"


zip_ref = zipfile.ZipFile('/tmp/data.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-06-10 10:08:59--  https://github.com/PolMix/nems_ai/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/PolMix/nems_ai/zip/refs/heads/main [following]
--2023-06-10 10:08:59--  https://codeload.github.com/PolMix/nems_ai/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/data.zip’

/tmp/data.zip           [      <=>           ]  15.86M  8.47MB/s    in 1.9s    

2023-06-10 10:09:01 (8.47 MB/s) - ‘/tmp/data.zip’ saved [16625899]



In [ ]:
%cd /tmp/nems_ai-main/Data
filename = 'Dataset.txt'
df = pd.read_fwf(filename)

/tmp/nems_ai-main/Data


Смотрим на содержимое датасета:

In [ ]:
df.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Pretension (Pa),Eigenfrequency (Hz),Quality factor (1),Effective mass,Noise (kg^2 / s^3),TED (W)
0,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,6.415102308860925E7+2400.623397410659i,1.336133e+04,2.362616e-15,1.051171e-22,2.602258e-12
1,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,1.6011837483039114E8+6638.939975583381i,1.205903e+04,2.589984e-15,3.186781e-22,2.991534e-12
2,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,2.909667084628352E8+12724.292499287933i,1.143351e+04,2.560363e-15,6.037978e-22,3.409430e-12
3,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,3.96451977228644E8+19548.92557221195i,1.013999e+04,8.780197e-16,3.181145e-22,4.959458e-12
4,105.195552,890.696186,209.252268,37.407275,3.716553,314.636835,30.305477,5.090849e+08,2569354.395333146+0.17898817427683494i,7.177442e+06,3.216064e-14,2.135344e-28,7.976759e-12


# Обработка датасета

Импорт необходимых функций:

In [ ]:
%cd '/tmp/nems_ai-main/Scripts'
from dataset_processing import row_col_transform, name_columns
from dataset_processing import return_neg_frequencies_index, del_frequencies
from dataset_processing import del_im_frequency, convert_freq_to_float
from dataset_processing import set_param_limits, catch_limit_overflow

/tmp/nems_ai-main/Scripts


Преобразование 4row x 1col --> 1row x 4col:

In [ ]:
num_common = 8

data = row_col_transform(df, num_common)

Форматирование названий колонок:

In [ ]:
data = name_columns(df, data, num_common)

Вывод информации об аномалиях мнимой части резонансной частоты, их удаление:

In [ ]:
# Детектирование аномалий:
anomaly_rows, anomaly_cols = return_neg_frequencies_index(data)
print("Row and column indices of detected anomalies of imaginary part of resonant frequency:")
print(anomaly_rows, anomaly_cols)
print(f"The length of the dataset is {data.shape[0]}")

# Удаление аномалий:
data = del_frequencies(data, anomaly_rows)
lst_i, lst_j = return_neg_frequencies_index(data)
print(f"Anomalies have been deleted, the length of the dataset is {data.shape[0]}")

Row and column indices of detected anomalies of imaginary part of resonant frequency:
[128, 176, 193, 198, 268, 301, 325, 343, 345, 350, 431, 463, 468, 476, 496, 498, 503, 507, 529, 592, 593, 630, 682, 692, 744, 755, 808, 827, 832, 833, 865, 894, 896, 972, 1007, 1052, 1080, 1217, 1313, 1315, 1322, 1336, 1346, 1367, 1380, 1406, 1476, 1491, 1529, 1539, 1545, 1603, 1610, 1611, 1613, 1652, 1663, 1673, 1693, 1721, 1729, 1743, 1789, 1824, 1830, 1844, 1845, 1871, 1877, 1956, 1984, 2071, 2081, 2083, 2100, 2115, 2124, 2181, 2189, 2277, 2312, 2316, 2343, 2360, 2411, 2430, 2506, 2544, 2598, 2654, 2888, 2903, 2909, 2947, 2990, 3078, 3111, 3232, 3252, 3362, 3421, 3539, 3555, 3569, 3642, 3655, 3673, 3694, 3701, 3715, 3797, 3898, 4013, 4027, 4075, 4079, 4159, 4185, 4220, 4236, 4240, 4313, 4319, 4323, 4336, 4393, 4410, 4434, 4447, 4487, 4504, 4510, 4520, 4529, 4537, 4542, 4584, 4611, 4633, 4667, 4701, 4752, 4840, 4893, 4948, 4953, 5000, 5095, 5101, 5140, 5153, 5243, 5277, 5283, 5287, 5297, 128, 193, 1

Удаление мнимой части всех элементов датасета:

In [ ]:
data = del_im_frequency(data)

In [ ]:
data = convert_freq_to_float(data)

Смотрим на статистику по параметрам перед указанием пределов для параметров:

In [ ]:
data.describe()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Pretension (Pa),M1 Eigenfrequency (Hz),M1 Quality factor (1),...,M3 Eigenfrequency (Hz),M3 Quality factor (1),M3 Effective mass,M3 Noise (kg^2 / s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor (1),M4 Effective mass,M4 Noise (kg^2 / s^3),M4 TED (W)
count,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4.909000e+03,4.909000e+03,4.909000e+03,...,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03
mean,195.469415,529.097516,150.819856,30.404340,23.742353,349.708484,17.227796,4.712346e+08,6.988914e+06,2.502643e+07,...,2.804685e+07,2.219908e+07,2.808115e-14,2.014207e-23,7.943504e-12,4.173375e+07,2.115728e+07,2.806848e-14,1.691331e-23,1.017113e-11
std,238.666890,269.281493,41.982835,11.712403,49.683629,145.560699,12.786909,3.190863e+08,1.098401e+07,8.329148e+07,...,5.009151e+07,7.344745e+07,4.388424e-14,3.843014e-22,7.561086e-12,7.418109e+07,8.666440e+07,4.391170e-14,2.545723e-22,9.704030e-12
min,10.006357,50.114701,66.096716,10.055810,0.010004,100.222269,0.000000,0.000000e+00,3.840254e+04,1.409866e+03,...,1.153109e+05,1.409853e+03,1.263636e-16,8.801085e-34,3.347321e-14,1.538710e+05,1.409842e+03,1.266484e-16,9.123322e-34,7.378453e-14
25%,36.605352,298.766024,107.242205,20.117549,0.309281,220.216016,5.140889,2.366180e+08,8.730700e+05,2.943045e+05,...,2.650417e+06,2.537637e+05,3.638416e-15,1.125043e-29,2.898015e-12,3.557429e+06,2.296537e+05,3.614451e-15,1.998218e-29,3.819665e-12
50%,97.149246,526.461810,150.537470,30.610468,2.657895,349.938323,17.005210,4.777992e+08,2.470389e+06,1.872628e+06,...,7.744980e+06,1.620164e+06,1.061271e-14,9.200261e-28,5.521390e-12,1.059702e+07,1.356222e+06,1.062533e-14,1.503564e-27,7.042181e-12
75%,257.772109,758.297463,193.682337,40.815722,19.179475,476.299101,28.608417,7.477595e+08,7.668586e+06,1.221293e+07,...,2.693589e+07,1.071371e+07,3.272321e-14,5.545136e-26,1.055391e-11,3.932619e+07,9.048577e+06,3.280904e-14,8.259448e-26,1.334680e-11
max,1195.787212,999.773967,234.883842,49.992126,299.880837,599.954225,39.995581,9.994946e+08,8.115660e+07,1.467439e+09,...,3.675392e+08,1.207506e+09,4.327311e-13,1.952995e-20,7.927833e-11,4.399336e+08,2.734598e+09,4.327028e-13,9.359624e-21,1.112834e-10


Задаем ограничения на значения параметров:

In [ ]:
# Раскомментируйте для применения пределов для параметров
param_limits = {#'Eigenfrequency (Hz)': [0, 3e7],
                #'Quality factor': [0, 5e8],
                #'TED (W)': [1e-20, 1]
                }

print('Dataset length before setting limits is {length}'.format(length=data.shape[0]))
data_lim = set_param_limits(data, param_limits)
print('Dataset length after setting limits is {length}'.format(length=data_lim.shape[0]))

Dataset length before setting limits is 4909
Dataset length after setting limits is 4909


In [ ]:
data_lim.describe()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Pretension (Pa),M1 Eigenfrequency (Hz),M1 Quality factor (1),...,M3 Eigenfrequency (Hz),M3 Quality factor (1),M3 Effective mass,M3 Noise (kg^2 / s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor (1),M4 Effective mass,M4 Noise (kg^2 / s^3),M4 TED (W)
count,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4.909000e+03,4.909000e+03,4.909000e+03,...,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03
mean,195.469415,529.097516,150.819856,30.404340,23.742353,349.708484,17.227796,4.712346e+08,6.988914e+06,2.502643e+07,...,2.804685e+07,2.219908e+07,2.808115e-14,2.014207e-23,7.943504e-12,4.173375e+07,2.115728e+07,2.806848e-14,1.691331e-23,1.017113e-11
std,238.666890,269.281493,41.982835,11.712403,49.683629,145.560699,12.786909,3.190863e+08,1.098401e+07,8.329148e+07,...,5.009151e+07,7.344745e+07,4.388424e-14,3.843014e-22,7.561086e-12,7.418109e+07,8.666440e+07,4.391170e-14,2.545723e-22,9.704030e-12
min,10.006357,50.114701,66.096716,10.055810,0.010004,100.222269,0.000000,0.000000e+00,3.840254e+04,1.409866e+03,...,1.153109e+05,1.409853e+03,1.263636e-16,8.801085e-34,3.347321e-14,1.538710e+05,1.409842e+03,1.266484e-16,9.123322e-34,7.378453e-14
25%,36.605352,298.766024,107.242205,20.117549,0.309281,220.216016,5.140889,2.366180e+08,8.730700e+05,2.943045e+05,...,2.650417e+06,2.537637e+05,3.638416e-15,1.125043e-29,2.898015e-12,3.557429e+06,2.296537e+05,3.614451e-15,1.998218e-29,3.819665e-12
50%,97.149246,526.461810,150.537470,30.610468,2.657895,349.938323,17.005210,4.777992e+08,2.470389e+06,1.872628e+06,...,7.744980e+06,1.620164e+06,1.061271e-14,9.200261e-28,5.521390e-12,1.059702e+07,1.356222e+06,1.062533e-14,1.503564e-27,7.042181e-12
75%,257.772109,758.297463,193.682337,40.815722,19.179475,476.299101,28.608417,7.477595e+08,7.668586e+06,1.221293e+07,...,2.693589e+07,1.071371e+07,3.272321e-14,5.545136e-26,1.055391e-11,3.932619e+07,9.048577e+06,3.280904e-14,8.259448e-26,1.334680e-11
max,1195.787212,999.773967,234.883842,49.992126,299.880837,599.954225,39.995581,9.994946e+08,8.115660e+07,1.467439e+09,...,3.675392e+08,1.207506e+09,4.327311e-13,1.952995e-20,7.927833e-11,4.399336e+08,2.734598e+09,4.327028e-13,9.359624e-21,1.112834e-10


Дополнительно: еще раз изменим тип всех данных на `float`:

# Сохраним датасет на диск:

На всякий случай ещё раз переименовываем названия столбцов:

In [ ]:
data_lim.columns = ['Beam length (um)', 'Beam width (nm)', 'Thickness_1 (nm)', 'Thickness_2 (nm)',
                    'Temperature (K)', 'Distance (nm)', 'Gate voltage (V)', 'Pretension (Pa)',
                    'M1 Eigenfrequency (Hz)', 'M1 Quality factor', 'M1 Effective mass (kg)', 'M1 Noise (kg^2/s^3)', 'M1 TED (W)',
                    'M2 Eigenfrequency (Hz)', 'M2 Quality factor', 'M2 Effective mass (kg)', 'M2 Noise (kg^2/s^3)', 'M2 TED (W)',
                    'M3 Eigenfrequency (Hz)', 'M3 Quality factor', 'M3 Effective mass (kg)', 'M3 Noise (kg^2/s^3)', 'M3 TED (W)',
                    'M4 Eigenfrequency (Hz)', 'M4 Quality factor', 'M4 Effective mass (kg)', 'M4 Noise (kg^2/s^3)', 'M4 TED (W)']

In [ ]:
data_lim.to_csv('Dataset_Processed.csv')

# Добавление вклада термоупругих потерь

Подмешаем к добротности в датасете добротность согласно приближению Roukes:

In [ ]:
# Fundamental properties:
rho = [3100, 2700] # density of matter
E = [250e9, 70e9] # Young's modulus
k = [20, 237] # Thermal conductivity constant
alpha = [2.3e-6, 23.1e-6] # Thermal expansion coefficient

# Calculated properties:
Cp = [700, 904] # heat capacity at constant pressure

In [ ]:
def calculate_Q_TED(df):
    param_history = {'Cp_av': [], 'E_av': [], 'rho_av': [], 'k_av': [], 'alpha_av': [], 'xi_av': [], 'chi': [], 'Q_ted': []}
    for row_index in range(0, df.shape[0]):
        # Calculating average properties of beam:
        Cp_av = (Cp[0] * df.iloc[row_index, 2] + Cp[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        E_av = (E[0] * df.iloc[row_index, 2] + E[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        rho_av = (rho[0] * df.iloc[row_index, 2] + rho[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        k_av = (k[0] * df.iloc[row_index, 2] + k[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        alpha_av = (alpha[0] * df.iloc[row_index, 2] + alpha[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        xi_av = k_av / (rho_av * Cp_av)
        param_history['Cp_av'].append(Cp_av)
        param_history['E_av'].append(E_av)
        param_history['rho_av'].append(rho_av)
        param_history['k_av'].append(k_av)
        param_history['alpha_av'].append(alpha_av)
        param_history['xi_av'].append(xi_av)

        for mode_number in range(1, 5):
            # Calculating TED quality factor:
            chi = df.iloc[row_index, 1] * 1e-9 * np.sqrt(2 * 3.1415 * df.loc[df.index[row_index], f'M{mode_number} Eigenfrequency (Hz)'] / (2 * xi_av))
            delta_ted = (E_av * alpha_av **2 * df.iloc[row_index, 4]) / (Cp_av * rho_av) * (6/(chi**2) - 6 * (np.sinh(chi) + np.sin(chi))/(chi**3 * (np.cosh(chi) + np.cos(chi))))
            df.loc[df.index[row_index], f'M{mode_number} Quality factor'] = 1/(1/df.loc[df.index[row_index], f'M{mode_number} Quality factor'] + delta_ted)

            param_history['chi'].append(chi)
            param_history['Q_ted'].append(1/delta_ted)

    return df, param_history

In [ ]:
print(data_catched.shape[0])
data_ted, history = calculate_Q_TED(data_catched)

4909


KeyError: ignored

In [ ]:
sorted_history = sorted(history['Q_ted'])
print(sorted_history)

In [ ]:
data_ted.head()